In [1]:
import pandas as pd
import numpy as np
import random
import dask # for faster import
import dask.dataframe as dd 
import dask.array as da
from datetime import datetime
import gc
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Loading the full datasets

In [3]:
path = '../../'

col_to_use = ['STORE_ID', 'DATE', 'INDIV_ID',
       'VEHICLE_ID', 'UNITS', 'SALES', 'STATE_CODE', 'ZIP_CODE', 'MAKE',
       'MODEL', 'SUB_MODEL', 'MODEL_YEAR', 'PROD_GROUP_CODE',
       'PROD_GROUP_DESC', 'CATEGORY_DESC',
       'SEGMENT_DESC', 'CLASS_DESC', 'DISCOUNT_FLAG',
       'CROSS_SECTION', 'ASPECT_RATIO', 'RIM_SIZE']

faster import for a large dataset

In [4]:
dy = pd.DataFrame()

#we are going to work with chunks of size 1 million rows
chunksize = 10 ** 6

for chunk in tqdm(pd.read_csv(path + 'JanOct16_trimmed.csv', chunksize=chunksize)):
    filtered = (chunk[(np.where(chunk['DISCOUNT_FLAG']!='Y', True, False))])
    dy = pd.concat([dy, filtered], ignore_index=True)

68it [10:05,  8.90s/it]


In [5]:
# the random sample is skipping 999 rows and taking 1 row on average 
# dy15 = pd.read_csv(path + 'AprOct15_trimmed.csv', usecols = col_to_use)#, skiprows = lambda i: i>0 and random.random() > 0.1)

In [6]:
# dy16 = pd.read_csv(path + 'JanOct16_trimmed.csv', usecols = col_to_use) 

In [7]:
# dy17 = pd.read_csv(path + 'JanOct17_trimmed.csv', usecols = col_to_use)

In [8]:
# dy18 = pd.read_csv(path + 'JanOct18_trimmed.csv', usecols = col_to_use)

In [9]:
dn = pd.read_csv(path + 'Nov16_trimmed.csv', usecols = col_to_use)

In [10]:
# dy = pd.concat([dy15, dy16, dy17])

In [11]:
# dy.to_csv(path + 'all_years.csv')

In [12]:
# ## using dask instead of pandas to fasten dataset import
# dtype={'ASPECT_RATIO': 'object',
#        'CROSS_SECTION': 'object',
#        'RIM_SIZE': 'object'}

In [13]:
# dy = dd.read_csv(path + 'all_years.csv', dtype = dtype)
# dy = pd.read_csv(path + 'all_years.csv', skiprows = lambda i: i>0 and random.random() > 0.0001)

# dy = pd.read_csv(path + 'all_years.csv', nrows = 1000)

In [14]:
dy.head()

,Unnamed: 0,Unnamed: 0.1,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,STATE_CODE,ZIP_CODE,MSA,MAKE,MODEL,SUB_MODEL,MODEL_YEAR,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_CODE,CATEGORY_DESC,SEGMENT_CODE,SEGMENT_DESC,CLASS_CODE,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE,EMAIL_OPTIN_IND,AH1_RES_BUS_INDC,SUPP1_BUS_PANDER
0,0,0,20869,991975080,2016-01-17,7013632,277902102.0,945291489,0.0,51.96,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,147.0,Balance,N,NaN,NaN,NaN,Y,R,N
1,1,1,20869,991975080,2016-01-17,7015016,277902102.0,945291489,0.0,0.00,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,139.0,Tire Mounting,N,NaN,NaN,NaN,Y,R,N
2,2,2,20869,991975080,2016-01-17,7097782,277902102.0,945291489,0.0,63.80,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,5.0,Tires,20158.0,Road Hazard,20159.0,Road Hazard,20160.0,Road Hazard,N,NONE,NONE,NONE,Y,R,N
3,3,3,20869,991975080,2016-01-17,7004578,277902102.0,945291489,0.0,84.99,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,4.0,Services,83.0,Steering/Suspension/Drivetrain,96.0,Alignments,49083.0,Alignment Service,N,NaN,NaN,NaN,Y,R,N
4,4,4,20869,991975080,2016-01-17,2809,277902102.0,945291489,4.0,373.96,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,5.0,Tires,26.0,Passenger Tires,30.0,P Metric Light Truck Tires,42.0,All Terrain Tires,N,265,70,16,Y,R,N


In [15]:
dy.shape

(40899870, 32)

## setting the new dataframe

In [16]:
vehicles = dy.drop_duplicates(subset = 'VEHICLE_ID',keep='first')

In [17]:
targets = dn[dn['PROD_GROUP_CODE'] == 5]['INDIV_ID'].unique()

In [18]:
unique_IDs = dy.INDIV_ID.unique()

In [19]:
year_indivs = pd.DataFrame(unique_IDs, columns = ['indiv'])

In [20]:
year_indivs.head()

,indiv
0,277902102.0
1,261419318.0
2,279504183.0
3,261790895.0
4,313827641.0


In [21]:
year_indivs['label'] = np.where(year_indivs['indiv'].isin(targets), 1, 0)

In [22]:
dy.shape

(40899870, 32)

In [23]:
year_indivs['cust_in_top_10_sales'] = np.where(year_indivs['indiv'].isin(dy.INDIV_ID.value_counts().nlargest(int(len(year_indivs)/10))),1,0)

In [24]:
year_indivs.label.value_counts()

0    2904871
1      96322
Name: label, dtype: int64

In [25]:
len(year_indivs)

3001193

### Feature : is the store among the top performers regarding total number of transactions in general?

In [26]:
top_stores_trans_list = dy.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [27]:
len(top_stores_trans_list)

2205

get individuals visiting those stores

In [28]:
indiv_store_list = dy[dy['STORE_ID'].isin(top_stores_trans_list)]['INDIV_ID']

In [29]:
year_indivs['store_top_tran'] = np.where(year_indivs.indiv.isin(indiv_store_list), 1, 0)

In [30]:
# sanity check
year_indivs['store_top_tran'].value_counts()

1    3001193
Name: store_top_tran, dtype: int64

In [31]:
del top_stores_trans_list
del indiv_store_list

gc.collect()

160

### Feature : is the store among the top performers regarding total number of TIRES transactions ?

In [32]:
tires = dy.query('PROD_GROUP_CODE == 5')

In [33]:
top_stores_tires_list = tires.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [34]:
top_stores_tires_list

Int64Index([237119, 240187, 326506, 517585, 517895, 323022, 517909, 517658,
            303599, 241666,
            ...
            779903, 779902,  27499,  17191, 779770, 595594,  23876,  24732,
            227573,  13439],
           dtype='int64', length=2204)

get individuals visiting those stores

In [35]:
indiv_store_tires_list = dy[dy['STORE_ID'].isin(top_stores_tires_list)]['INDIV_ID']

In [36]:
year_indivs['store_top_tires_tran'] = np.where(year_indivs.indiv.isin(indiv_store_tires_list), 1, 0)

In [37]:
# sanity check
year_indivs['store_top_tires_tran'].value_counts()

1    3001185
0          8
Name: store_top_tires_tran, dtype: int64

In [38]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran
0,277902102.0,0,0,1,1
1,261419318.0,0,0,1,1
2,279504183.0,0,0,1,1
3,261790895.0,0,0,1,1
4,313827641.0,0,0,1,1


In [39]:
del indiv_store_tires_list
del top_stores_tires_list

gc.collect()

60

## Feature: has this vehicle recently had a tire service? 

consider adding a certain time limit 

### list of vehicles with recent tires' services 

Here I will remove VEHICLE_ID = 1   These cars don't have MAKE, MODEL, MODEL YEAR. it seems that data is missing regarding the vehicle description

In [40]:
vhc_tire_svc_array = dy.query('CATEGORY_DESC == "Tire Services" & VEHICLE_ID != 1')['VEHICLE_ID'].unique()

In [41]:
indiv_vhc_svc_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_svc_array)]['INDIV_ID']

In [42]:
year_indivs['vehicle_tire_svc'] = np.where(year_indivs.indiv.isin(indiv_vhc_svc_list), 1, 0)

In [43]:
# sanity check
year_indivs['vehicle_tire_svc'].value_counts()

1    1985475
0    1015718
Name: vehicle_tire_svc, dtype: int64

### How many transactions did these individuals [who came at least once for tire services] have with bridgestone before?

In [44]:
number_trans_for_ind_with_tire_services = indiv_vhc_svc_list.value_counts().reset_index()

number_trans_for_ind_with_tire_services.columns = ['indiv', 'number_trans']
# year_indivs['number_trans_for_ind_with_tire_services'] = np.where(year_indivs.indiv.isin(number_trans_for_ind_with_tire_services.index),
#                                                                   number_trans_for_ind_with_tire_services.INDIV_ID, 0)

number_trans_for_ind_with_tire_services.set_index("indiv")
# number_trans_for_ind_with_tire_services.head()
year_indivs = year_indivs.merge(number_trans_for_ind_with_tire_services, left_on = 'indiv', right_on = 'indiv', how = 'left')


In [45]:
year_indivs.number_trans = year_indivs.number_trans.fillna(0)

In [46]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans
0,277902102.0,0,0,1,1,1,10.0
1,261419318.0,0,0,1,1,1,5.0
2,279504183.0,0,0,1,1,1,16.0
3,261790895.0,0,0,1,1,0,0.0
4,313827641.0,0,0,1,1,0,0.0


In [47]:
year_indivs.shape

(3001193, 7)

### Can we add only transaction if they occured different days? This may mean the number of visits

In [48]:
indiv_vhc_svc_with_dates = dy[dy['VEHICLE_ID'].isin(vhc_tire_svc_array)][['INDIV_ID', 'DATE']]

In [49]:
indiv_vhc_svc_with_dates.head()

,INDIV_ID,DATE
0,277902102.0,2016-01-17
1,277902102.0,2016-01-17
2,277902102.0,2016-01-17
3,277902102.0,2016-01-17
4,277902102.0,2016-01-17


In [50]:
number_visits_for_ind_with_tire_services = indiv_vhc_svc_with_dates.groupby('INDIV_ID').agg({'DATE': 'nunique'}).reset_index()

number_visits_for_ind_with_tire_services.columns = ['indiv', 'number_visits'] 

year_indivs = year_indivs.merge(number_visits_for_ind_with_tire_services, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [51]:
year_indivs.head(100)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits
0,277902102.0,0,0,1,1,1,10.0,2.0
1,261419318.0,0,0,1,1,1,5.0,2.0
2,279504183.0,0,0,1,1,1,16.0,1.0
3,261790895.0,0,0,1,1,0,0.0,NaN
4,313827641.0,0,0,1,1,0,0.0,NaN
5,252330701.0,0,0,1,1,1,32.0,4.0
6,289610479.0,0,0,1,1,1,79.0,9.0
7,274759571.0,0,0,1,1,0,0.0,NaN
8,319449747.0,0,0,1,1,0,0.0,NaN
9,298347284.0,0,0,1,1,1,14.0,3.0


In [52]:
year_indivs.number_visits = year_indivs.number_visits.fillna(0)

In [53]:
# sanity check
year_indivs.number_visits.equals(year_indivs.number_trans)

False

In [54]:
year_indivs.shape

(3001193, 8)

## Feature: how many times did the individual come to purchase tires in the past? I will consider trasnactions within the same day as one visit

In [55]:
tire_pur_indiv = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')[['INDIV_ID', 'DATE']]

In [56]:
tire_pur_indiv.head()

,INDIV_ID,DATE
2,277902102.0,2016-01-17
4,277902102.0,2016-01-17
66,298347284.0,2016-01-06
67,298347284.0,2016-01-06
84,267161356.0,2016-01-25


In [57]:
number_visits_tire_purch = tire_pur_indiv.groupby('INDIV_ID').agg({'DATE': 'nunique'}).reset_index()

number_visits_tire_purch.columns = ['indiv', 'number_visits_tire_purchase'] 

year_indivs = year_indivs.merge(number_visits_tire_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [58]:
year_indivs.number_visits_tire_purchase = year_indivs.number_visits_tire_purchase.fillna(0)

In [59]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0


In [60]:
year_indivs.shape

(3001193, 9)

## Feature: has anyone purchased a tire for this vehicle before?

we need to settle on the definition of tire purchase, is it 5 only or after removing others as well?

In [61]:
vhc_tire_pur_array = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others" & VEHICLE_ID != 1')['VEHICLE_ID'].unique()

In [62]:
vhc_tire_pur_array

array([945291489, 936895032, 945422040, ..., 952382299, 951698839,
       952382302])

In [63]:
indiv_vhc_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_pur_array)]['INDIV_ID']

In [64]:
year_indivs['vhc_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_pur_list), 1, 0)

In [65]:
# sanity check
year_indivs['vhc_tire_purchase'].value_counts()

0    1870015
1    1131178
Name: vhc_tire_purchase, dtype: int64

## Feature : when was the last time this vehicle bought a new tire? [last year - 2 years - 3 years]

In [66]:
dy['DATE'] = pd.to_datetime(dy['DATE'])

In [67]:
dy['year'] = pd.DatetimeIndex(dy['DATE']).year
dy['month'] = pd.DatetimeIndex(dy['DATE']).month

In [68]:
dy_vhc_tire_purch = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')

In [69]:
month_look = dy_vhc_tire_purch.groupby('VEHICLE_ID').agg({'month':['max'] })

Max is the latest month the tire was purchased. If the last month to purchase a tire was before 6, the vehicle may need to replace tires. 
Of course, this needs a change, but after we combine datasets of different years. 

In [70]:
month_look.columns = ['last_mo_tire_purch']

In [71]:
vhc_early_purch_tires = month_look.reset_index().query('last_mo_tire_purch < 6')['VEHICLE_ID']

In [72]:
indiv_vhc_early_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_early_purch_tires)]['INDIV_ID']

In [73]:
year_indivs['vhc_early_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_early_pur_list), 1, 0)

In [74]:
# sanity check
year_indivs['vhc_early_tire_purchase'].value_counts()

0    2441815
1     559378
Name: vhc_early_tire_purchase, dtype: int64

## Feature : when was the last time this vehicle bought a new tire? (numerical) 
Reference point is Oct 17 for training and Oct 18 for testing. 
This will be automatic by getting max date

For example, the value should be 3 if purchased 3 months ago. 

In [75]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [76]:
d1 = dy.DATE.max()

In [77]:
last_purch_per_indiv = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'DATE':'max'}).reset_index()

last_purch_per_indiv.columns = ['indiv', 'last_tire_purchase'] 

year_indivs = year_indivs.merge(last_purch_per_indiv, left_on = 'indiv', right_on = 'indiv', how = 'left')


year_indivs['diff_from_last_tire_purch'] = diff_month(d1, pd.DatetimeIndex(year_indivs.last_tire_purchase))

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN


### Here I need to understand what to do with units 

In [78]:
dy_vhc_tire_purch.head()  # dataset created before for tires purchased 

,Unnamed: 0,Unnamed: 0.1,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,STATE_CODE,ZIP_CODE,MSA,MAKE,MODEL,SUB_MODEL,MODEL_YEAR,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_CODE,CATEGORY_DESC,SEGMENT_CODE,SEGMENT_DESC,CLASS_CODE,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE,EMAIL_OPTIN_IND,AH1_RES_BUS_INDC,SUPP1_BUS_PANDER,year,month
2,2,2,20869,991975080,2016-01-17,7097782,277902102.0,945291489,0.0,63.80,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,5.0,Tires,20158.0,Road Hazard,20159.0,Road Hazard,20160.0,Road Hazard,N,NONE,NONE,NONE,Y,R,N,2016,1
4,4,4,20869,991975080,2016-01-17,2809,277902102.0,945291489,4.0,373.96,TN,37221,"NASHVILLE,",GMC,K1500 SUBURBAN SLE,SLE,1997.0,5.0,Tires,26.0,Passenger Tires,30.0,P Metric Light Truck Tires,42.0,All Terrain Tires,N,265,70,16,Y,R,N,2016,1
66,200,200,317902,991323280,2016-01-06,7097782,298347284.0,936895032,0.0,7.50,TX,76548,KILLEEN-TE,NISSAN,VERSA 1.6 S,1.6 S,2014.0,5.0,Tires,20158.0,Road Hazard,20159.0,Road Hazard,20160.0,Road Hazard,N,NONE,NONE,NONE,Y,R,N,2016,1
67,201,201,317902,991323280,2016-01-06,123905,298347284.0,936895032,1.0,48.99,TX,76548,KILLEEN-TE,NISSAN,VERSA 1.6 S,1.6 S,2014.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,35.0,Economy Tires,N,185,65,15,Y,R,N,2016,1
84,241,241,307228,991306240,2016-01-25,7097782,267161356.0,945422040,0.0,20.00,TN,38125,"MEMPHIS,TN",CHEVROLET,EXPRESS 1500 BASE,NaN,2006.0,5.0,Tires,20158.0,Road Hazard,20159.0,Road Hazard,20160.0,Road Hazard,N,NONE,NONE,NONE,Y,R,N,2016,1


In [79]:
dy.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'STORE_ID', 'TRAN_ID', 'DATE',
       'ARTICLE_ID', 'INDIV_ID', 'VEHICLE_ID', 'UNITS', 'SALES', 'STATE_CODE',
       'ZIP_CODE', 'MSA', 'MAKE', 'MODEL', 'SUB_MODEL', 'MODEL_YEAR',
       'PROD_GROUP_CODE', 'PROD_GROUP_DESC', 'CATEGORY_CODE', 'CATEGORY_DESC',
       'SEGMENT_CODE', 'SEGMENT_DESC', 'CLASS_CODE', 'CLASS_DESC',
       'DISCOUNT_FLAG', 'CROSS_SECTION', 'ASPECT_RATIO', 'RIM_SIZE',
       'EMAIL_OPTIN_IND', 'AH1_RES_BUS_INDC', 'SUPP1_BUS_PANDER', 'year',
       'month'],
      dtype='object')

In [80]:
dy_vhc_tire_purch.groupby(['CATEGORY_DESC', 'SEGMENT_DESC', 'CLASS_DESC'], dropna = False).agg({'UNITS': ['min', 'nunique']})

UNITS  \
                                                                                       min   
CATEGORY_DESC     SEGMENT_DESC                     CLASS_DESC                                
Light Truck Tires Commercial Tires                 All Terrain Tires                  -6.0   
                                                   Highway Tires                      -6.0   
                  Commercial/Recreational          All-Terrain                         0.0   
                                                   Highway All-Season                  0.0   
                                                   Maximum Traction                    0.0   
                  Recreational Tires               All Terrain Tires                  -6.0   
                                                   Highway Tires                      -4.0   
                                                   Max Traction Tires                 -5.0   
                  Snow Tires-LT                    Snow Tires - All Terrain LT         1.0   
                                                   Snow Tires - Commercial LT          1.0   
Others            Agricultural Tires               Garden Tires                        0.0   
                                                   Rear Ag Tires                       2.0   
                  Medium Truck Tires               Radial Tires                        1.0   
                  Miscellaneous Tire Related Items Misc/Discounts/Non-Inventoreable   -4.0   
                                                   Other                              -4.0   
                  Tubes                            Tubes                              -5.0   
Passenger Tires   Other Passenger Tires            Other Passenger Tires             -20.0   
                  P Metric Light Truck Tires       All Terrain Tires                  -5.0   
                                                   Highway Tires                      -8.0   
                  P-Metric CUV/SUV                 CUV/SUV All-Terrain                 1.0   
                                                   CUV/SUV Highway All-Season         -8.0   
                                                   CUV/SUV Sport Performance Summer    0.0   
                  Performance Tires                Performance All-Season             -4.0   
                                                   S/T Performance Tires              -4.0   
                                                   Touring H/V/Z Tires                -6.0   
                                                   Ultra High Performance All Season  -4.0   
                                                   Ultra High Performance Summer      -4.0   
                  Snow Tires-PS                    Snow Tires                         -4.0   
                  Touring                          Car & Minivan All-Season           -4.0   
                                                   Economy Tires                     -10.0   
                                                   Mass Market Tires                  -4.0   
Road Hazard       Road Hazard                      Road Hazard                         0.0   

                                                                                              
                                                                                     nunique  
CATEGORY_DESC     SEGMENT_DESC                     CLASS_DESC                                 
Light Truck Tires Commercial Tires                 All Terrain Tires                      13  
                                                   Highway Tires                          21  
                  Commercial/Recreational          All-Terrain                             5  
                                                   Highway All-Season                      1  
                                                   Maximum Traction                        4  
                  Recreational Tir

It seems that zeros in the units do actually make sense. 

## For those who purchased tires, how many units did they purchase for all years?

In [81]:
units_tires_purch = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'UNITS':'sum'}).reset_index()

units_tires_purch.columns = ['indiv', 'units_tires_purch'] 

year_indivs = year_indivs.merge(units_tires_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN


In [82]:
year_indivs.shape

(3001193, 14)

## For those who purchased tires, what is their total sales for all years?

In [83]:
sales_tires_purch = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'SALES':'sum'}).reset_index()

sales_tires_purch.columns = ['indiv', 'sales_tires_purch'] 

year_indivs = year_indivs.merge(sales_tires_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN


In [84]:
year_indivs.shape

(3001193, 15)

In [85]:
# year_indivs.to_csv(path + 'curated_data/part1_full_data.csv'))

## Feature: top customers regarding overall visits  
I assume visits occur on separate dates

In [86]:
number_trans_general = dy.groupby('INDIV_ID').agg({'DATE':'nunique'}).reset_index()

number_trans_general.columns = ['indiv', 'number_trans_general'] 

year_indivs = year_indivs.merge(number_trans_general, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6


In [87]:
year_indivs.shape

(3001193, 16)

In [88]:
# top_cust_overall_list = dy.INDIV_ID.value_counts().nlargest(int(len(dy)/10))

In [89]:
# year_indivs['top_cust_overall'] = np.where(year_indivs['indiv'].isin(top_cust_overall_list), 1, 0)

In [90]:
## Feature: top customers regarding tires' purchase 
# top_cust_tire_purch_array = dy_vhc_tire_purch.INDIV_ID.value_counts().nlargest(int(len(dy_vhc_tire_purch)/10)).index
# year_indivs['top_cust_tire_purch'] = np.where(year_indivs['indiv'].isin(top_cust_tire_purch_array), 1, 0)
# year_indivs['top_cust_tire_purch'].value_counts()
# year_indivs.head(10)

## Feature: Top makes based on number of times a tire is purchased 

In [91]:
top_make_tire_purch_array = dy_vhc_tire_purch.MAKE.value_counts().nlargest(int(len(dy_vhc_tire_purch)/20)).index.tolist()[0:10]

In [92]:
top_make_tire_purch_array

['TOYOTA',
 'FORD',
 'CHEVROLET',
 'HONDA',
 'NISSAN',
 'DODGE',
 'HYUNDAI',
 'KIA',
 'CHRYSLER',
 'MAZDA']

In [93]:
dy['MAKE'] = dy['MAKE'].astype('str')

In [94]:
indiv_make_tire_purch = dy[dy['MAKE'].isin(top_make_tire_purch_array)]['INDIV_ID']

In [95]:
year_indivs['indiv_make_tire_pch'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch), 1, 0)

In [96]:
year_indivs['indiv_make_tire_pch'].value_counts()

1    2253618
0     747575
Name: indiv_make_tire_pch, dtype: int64

In [97]:
year_indivs.head(10)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0
5,252330701.0,0,0,1,1,1,32.0,4.0,2.0,1,0,2016-10-23,0.0,4.0,369.62,4,0
6,289610479.0,0,0,1,1,1,79.0,9.0,1.0,1,0,2016-07-21,3.0,2.0,133.74,13,1
7,274759571.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,4,1
8,319449747.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1
9,298347284.0,0,0,1,1,1,14.0,3.0,1.0,1,1,2016-01-06,9.0,1.0,56.49,3,1


## Proportion of MAKE among all other MAKES

In [98]:
prop_make = dy.MAKE.value_counts(normalize = True).reset_index()

prop_make.columns = ['MAKE', 'make_prop']

In [99]:
prop_make.head()

,MAKE,make_prop
0,TOYOTA,0.132057
1,FORD,0.118258
2,CHEVROLET,0.112570
3,HONDA,0.102898
4,NISSAN,0.076643


I will assume here that each individual has a single car, which is the first car

In [100]:
dy_make = dy[['INDIV_ID', 'MAKE']]  #extract the columns ID and MAKE from original dataset - this can be used for the join

In [101]:
dy_make = dy_make.merge(prop_make, left_on = 'MAKE', right_on = 'MAKE', how = 'left')


In [102]:
dy_make.head()

,INDIV_ID,MAKE,make_prop
0,277902102.0,GMC,0.024105
1,277902102.0,GMC,0.024105
2,277902102.0,GMC,0.024105
3,277902102.0,GMC,0.024105
4,277902102.0,GMC,0.024105


In [103]:
dy_make = dy_make.drop('MAKE', axis = 1)
dy_make.columns = ['indiv', 'make_prop']

In [104]:
dy_make.shape

(40899870, 2)

In [105]:
dy_make = dy_make.drop_duplicates(subset = ['indiv'])

dy_make.shape

(3001193, 2)

In [106]:
year_indivs = year_indivs.merge(dy_make, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [107]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084


In [108]:
year_indivs.shape

(3001193, 18)

## Top MAKE based on proportion of tires purchased in comparision to total transactions

I Have to make sure that MAKE that were presented only once are ommitted

In [109]:
dy['tire_purchase'] = np.where((dy['PROD_GROUP_DESC'] == "Tires") & (dy['SEGMENT_DESC'] != "Others"), 1, 0)

df2 = dy.groupby(['MAKE', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 100')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

top_make_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MAKE'].tolist()[0:1000]
                                                                                                                                      
indiv_make_tire_purch_prop = dy[dy['MAKE'].isin(top_make_tires_purch_prop_array)]['INDIV_ID']

year_indivs['make_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch_prop), 1, 0)

year_indivs['make_prop_tire'].value_counts()

0    2745478
1     255715
Name: make_prop_tire, dtype: int64

## Proportion of MODEL among all other MODELS

In [110]:
prop_MODEL = dy.MODEL.value_counts(normalize = True).reset_index()

prop_MODEL.columns = ['MODEL', 'MODEL_prop']

In [111]:
prop_MODEL.head()

,MODEL,MODEL_prop
0,CAMRY LE,0.020440
1,ACCORD EX,0.016978
2,ALTIMA S,0.011920
3,CIVIC EX,0.010561
4,COROLLA LE,0.010214


I will assume here that each individual has a single car, which is the first car

In [112]:
dy_MODEL = dy[['INDIV_ID', 'MODEL']]  #extract the columns ID and MODEL from original dataset - this can be used for the join

In [113]:
dy_MODEL = dy_MODEL.merge(prop_MODEL, left_on = 'MODEL', right_on = 'MODEL', how = 'left')


In [114]:
dy_MODEL.head()

,INDIV_ID,MODEL,MODEL_prop
0,277902102.0,K1500 SUBURBAN SLE,0.000023
1,277902102.0,K1500 SUBURBAN SLE,0.000023
2,277902102.0,K1500 SUBURBAN SLE,0.000023
3,277902102.0,K1500 SUBURBAN SLE,0.000023
4,277902102.0,K1500 SUBURBAN SLE,0.000023


In [115]:
dy_MODEL = dy_MODEL.drop('MODEL', axis = 1)
dy_MODEL.columns = ['indiv', 'MODEL_prop']

In [116]:
dy_MODEL = dy_MODEL.drop_duplicates(subset = ['indiv'])

In [117]:
year_indivs = year_indivs.merge(dy_MODEL, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [118]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105,0,0.000023
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643,0,0.001021
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503,0,0.001486
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559,0,0.007031
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084,0,0.000089


In [119]:
year_indivs.shape

(3001193, 20)

## Top MODEL based on proportion of tires purchased in comparision to total transactions

In [120]:
df2 = dy.groupby(['MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 10')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

dy['MODEL'] = dy['MODEL'].astype('str')

top_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MODEL'].tolist()[0:1000]

indiv_MODEL_tire_purch_prop = dy[dy['MODEL'].isin(top_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                       
year_indivs['MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                       
year_indivs['MODEL_prop_tire'].value_counts()                                                                                                                                    

0    2861575
1     139618
Name: MODEL_prop_tire, dtype: int64

## Proportion of SUB_MODEL among all other SUB_MODELS

In [121]:
prop_SUB_MODEL = dy.SUB_MODEL.value_counts(normalize = True).reset_index()

prop_SUB_MODEL.columns = ['SUB_MODEL', 'SUB_MODEL_prop']

In [122]:
prop_SUB_MODEL = prop_SUB_MODEL.dropna(subset = ['SUB_MODEL'])

In [123]:
prop_SUB_MODEL.head()

,SUB_MODEL,SUB_MODEL_prop
0,Base,0.136945
1,EX,0.066980
2,SE,0.055736
3,Limited,0.042796
4,LX,0.039853


I will assume here that each individual has a single car, which is the first car

In [124]:
dy_SUB_MODEL = dy[['INDIV_ID', 'SUB_MODEL']]  #extract the columns ID and SUB_MODEL from original dataset - this can be used for the join

In [125]:
dy_SUB_MODEL = dy_SUB_MODEL.dropna(subset = ['SUB_MODEL'])

In [126]:
dy_SUB_MODEL = dy_SUB_MODEL.drop_duplicates(subset = ['SUB_MODEL'])

In [127]:
dy_SUB_MODEL = dy_SUB_MODEL.merge(prop_SUB_MODEL, left_on = 'SUB_MODEL', right_on = 'SUB_MODEL', how = 'left')

In [128]:
dy_SUB_MODEL.head()

,INDIV_ID,SUB_MODEL,SUB_MODEL_prop
0,277902102.0,SLE,0.006679
1,261419318.0,S,0.038920
2,279504183.0,EX,0.066980
3,261790895.0,Laredo,0.006402
4,313827641.0,GT2,0.000097


In [129]:
dy_SUB_MODEL = dy_SUB_MODEL.drop('SUB_MODEL', axis = 1)

dy_SUB_MODEL.columns = ['indiv', 'SUB_MODEL_prop']

In [130]:
dy_SUB_MODEL = dy_SUB_MODEL.dropna(subset = ['indiv'])
dy_SUB_MODEL.drop_duplicates(subset = ['indiv'], inplace = True)

In [131]:
year_indivs = year_indivs.merge(dy_SUB_MODEL, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [132]:
year_indivs.shape

(3001193, 22)

## Top SUB_MODEL based on proportion of tires purchased in comparision to total transactions

In [133]:
df2 = dy.groupby(['SUB_MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 10')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

dy['SUB_MODEL'] = dy['SUB_MODEL'].astype('str')

top_SUB_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10), 'total')['SUB_MODEL'].tolist()[0:1000]            

indiv_SUB_MODEL_tire_purch_prop = dy[dy['SUB_MODEL'].isin(top_SUB_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_SUB_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'].value_counts()

0    2905925
1      95268
Name: SUB_MODEL_prop_tire, dtype: int64

In [134]:
year_indivs.head(10)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105,0,0.000023,0,0.006679,0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643,0,0.001021,0,0.038920,0
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503,0,0.001486,0,0.066980,0
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559,0,0.007031,0,0.006402,0
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084,0,0.000089,0,0.000097,0
5,252330701.0,0,0,1,1,1,32.0,4.0,2.0,1,0,2016-10-23,0.0,4.0,369.62,4,0,0.014395,0,0.004469,0,0.136945,0
6,289610479.0,0,0,1,1,1,79.0,9.0,1.0,1,0,2016-07-21,3.0,2.0,133.74,13,1,0.028503,0,0.001393,0,0.039853,0
7,274759571.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,4,1,0.112570,0,0.001301,0,NaN,0
8,319449747.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132057,0,0.002842,0,0.042796,0
9,298347284.0,0,0,1,1,1,14.0,3.0,1.0,1,1,2016-01-06,9.0,1.0,56.49,3,1,0.076643,0,0.000837,0,0.000857,0


## SEGMENT Description

In [135]:
# func that returns a dummified DataFrame of significant dummies in a given column
def dum_sign(dummy_col, threshold=0.1, columns = None):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col.columns) / len(dummy_col.columns)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[columns][~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.columns.name, drop_first = True, columns = columns)

In [136]:
# dataset after filtering for tire purchases
dy_tire_segment = dy_vhc_tire_purch[['INDIV_ID','SEGMENT_DESC']]

dy_tire_segment = dum_sign(dy_tire_segment, columns = ['SEGMENT_DESC'])

dy_tire_segment = dy_tire_segment.rename(columns = {'INDIV_ID' : 'indiv'})

dy_tire_segment = dy_tire_segment.drop_duplicates(subset = ['indiv'])

year_indivs = year_indivs.merge(dy_tire_segment, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [137]:
year_indivs.head(200)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105,0,2.324006e-05,0,0.006679,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643,0,1.020902e-03,0,0.038920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503,0,1.486026e-03,0,0.066980,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559,0,7.031331e-03,0,0.006402,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084,0,8.944201e-05,0,0.000097,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,252330701.0,0,0,1,1,1,32.0,4.0,2.0,1,0,2016-10-23,0.0,4.0,369.62,4,0,0.014395,0,4.469325e-03,0,0.136945,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,289610479.0,0,0,1,1,1,79.0,9.0,1.0,1,0,2016-07-21,3.0,2.0,133.74,13,1,0.028503,0,1.393487e-03,0,0.039853,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,274759571.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,4,1,0.112570,0,1.301096e-03,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,319449747.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132057,0,2.842398e-03,0,0.042796,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,298347284.0,0,0,1,1,1,14.0,3.0,1.0,1,1,2016-01-06,9.0,1.0,56.49,3,1,0.076643,0,8.371623e-04,0,0.000857,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [138]:
year_indivs.shape

(3001193, 37)

## Individuals in top 5 segments 

In [139]:
segment_tire_sales = dy[dy['PROD_GROUP_CODE']==5].groupby(['SEGMENT_DESC'])['SALES'].count().to_frame()
segment_tire_sales.rename(columns = {'SALES': 'NUMBER_OF_TIRE_SALES'},inplace = True)
top_5_segment_tire_sales = segment_tire_sales.sort_values('NUMBER_OF_TIRE_SALES', ascending = False)[:5]
top_5_segment_tire_sales


,NUMBER_OF_TIRE_SALES
SEGMENT_DESC,
Road Hazard,1066437
Touring,584042
Performance Tires,521111
P Metric Light Truck Tires,224683
P-Metric CUV/SUV,201084


In [140]:
top_5_segment_tire_sales.reset_index(inplace = True)
top_5_segment_tire_sales = top_5_segment_tire_sales['SEGMENT_DESC']
indiv_top_5_segment_tire_sales = dy[dy['SEGMENT_DESC'].isin(top_5_segment_tire_sales)]['INDIV_ID']
year_indivs['top_5_segment'] = np.where(year_indivs.indiv.isin(indiv_top_5_segment_tire_sales), 1, 0)
year_indivs['top_5_segment'].value_counts()

0    1880192
1    1121001
Name: top_5_segment, dtype: int64

## CLASS Description

In [141]:
# dataset after filtering for tire purchases      # WILL BE used as a backup plan 
dy_tire_CLASS = dy_vhc_tire_purch[['INDIV_ID','CLASS_DESC']]

dy_tire_CLASS = dum_sign(dy_tire_CLASS, columns = ['CLASS_DESC'])

dy_tire_CLASS = dy_tire_CLASS.rename(columns = {'INDIV_ID' : 'indiv'})

dy_tire_CLASS = dy_tire_CLASS.drop_duplicates(subset = ['indiv'])

year_indivs = year_indivs.merge(dy_tire_CLASS, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


## Tire service

In [142]:
dy_tire_service = dy.query('CATEGORY_DESC == "Tire Services" & CLASS_DESC != "Discount Tire Services"')

In [143]:
# dy_tire_service.CLASS_DESC.value_counts()

In [144]:
dy_service = dy_tire_service[['INDIV_ID','CLASS_DESC']]

dy_service = dum_sign(dy_service, columns = ['CLASS_DESC'])

dy_service = dy_service.rename(columns = {'INDIV_ID' : 'indiv'})

dy_service.drop_duplicates(subset = ['indiv'], inplace = True)

year_indivs = year_indivs.merge(dy_service, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [145]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes,top_5_segment,CLASS_DESC_All-Terrain,CLASS_DESC_CUV/SUV All-Terrain,CLASS_DESC_CUV/SUV Highway All-Season,CLASS_DESC_CUV/SUV Sport Performance Summer,CLASS_DESC_Car & Minivan All-Season,CLASS_DESC_Economy Tires,CLASS_DESC_Garden Tires,CLASS_DESC_Highway All-Season,CLASS_DESC_Highway Tires,CLASS_DESC_Mass Market Tires,CLASS_DESC_Max Traction Tires,CLASS_DESC_Maximum Traction,CLASS_DESC_Misc/Discounts/Non-Inventoreable,CLASS_DESC_Other,CLASS_DESC_Other Passenger Tires,CLASS_DESC_Performance All-Season,CLASS_DESC_Radial Tires,CLASS_DESC_Rear Ag Tires,CLASS_DESC_Road Hazard,CLASS_DESC_S/T Performance Tires,CLASS_DESC_Snow Tires,CLASS_DESC_Snow Tires - All Terrain LT,CLASS_DESC_Snow Tires - Commercial LT,CLASS_DESC_Touring H/V/Z Tires,CLASS_DESC_Tubes,CLASS_DESC_Ultra High Performance All Season,CLASS_DESC_Ultra High Performance Summer,CLASS_DESC_Farm/Truck Services,CLASS_DESC_Misc. Tire Service,CLASS_DESC_Tire Mounting,CLASS_DESC_Tire Pressure Monitoring Systems,CLASS_DESC_Tire Rotation,CLASS_DESC_Valves
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105,0,0.000023,0,0.006679,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643,0,0.001021,0,0.038920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503,0,0.001486,0,0.066980,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559,0,0.007031,0,0.006402,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084,0,0.000089,0,0.000097,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
year_indivs.shape

(3001193, 71)

In [147]:
# year_indivs.to_csv('individuals18.csv')

In [148]:
dy = dy.replace('NONE', np.nan) # removing NONE from the 3 columns 

In [149]:
dy_tire_specs = dy[['INDIV_ID', 'VEHICLE_ID','CROSS_SECTION','ASPECT_RATIO', 'RIM_SIZE']]

In [150]:
dy_tire_specs.CROSS_SECTION = dy_tire_specs.CROSS_SECTION.astype('float')
dy_tire_specs.ASPECT_RATIO = dy_tire_specs.ASPECT_RATIO.astype('float')
dy_tire_specs.RIM_SIZE = dy_tire_specs.RIM_SIZE.astype('float')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [151]:
dy_tire_specs = dy_tire_specs.groupby('INDIV_ID').agg({'CROSS_SECTION' : 'min','ASPECT_RATIO': 'min'
                                                       , 'RIM_SIZE': 'min'})

In [152]:
dy_tire_specs = dy_tire_specs.reset_index()

In [153]:
dy_tire_specs = dy_tire_specs.rename(columns = {'INDIV_ID' : 'indiv'})

In [154]:
dy_service.drop_duplicates(subset = ['indiv'], inplace = True)


In [155]:
dy_tire_specs.head()

,indiv,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE
0,251634506.0,205.0,55.0,16.0
1,251634507.0,NaN,NaN,NaN
2,251634512.0,NaN,NaN,NaN
3,251634533.0,245.0,65.0,17.0
4,251634548.0,NaN,NaN,NaN


In [156]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes,top_5_segment,CLASS_DESC_All-Terrain,CLASS_DESC_CUV/SUV All-Terrain,CLASS_DESC_CUV/SUV Highway All-Season,CLASS_DESC_CUV/SUV Sport Performance Summer,CLASS_DESC_Car & Minivan All-Season,CLASS_DESC_Economy Tires,CLASS_DESC_Garden Tires,CLASS_DESC_Highway All-Season,CLASS_DESC_Highway Tires,CLASS_DESC_Mass Market Tires,CLASS_DESC_Max Traction Tires,CLASS_DESC_Maximum Traction,CLASS_DESC_Misc/Discounts/Non-Inventoreable,CLASS_DESC_Other,CLASS_DESC_Other Passenger Tires,CLASS_DESC_Performance All-Season,CLASS_DESC_Radial Tires,CLASS_DESC_Rear Ag Tires,CLASS_DESC_Road Hazard,CLASS_DESC_S/T Performance Tires,CLASS_DESC_Snow Tires,CLASS_DESC_Snow Tires - All Terrain LT,CLASS_DESC_Snow Tires - Commercial LT,CLASS_DESC_Touring H/V/Z Tires,CLASS_DESC_Tubes,CLASS_DESC_Ultra High Performance All Season,CLASS_DESC_Ultra High Performance Summer,CLASS_DESC_Farm/Truck Services,CLASS_DESC_Misc. Tire Service,CLASS_DESC_Tire Mounting,CLASS_DESC_Tire Pressure Monitoring Systems,CLASS_DESC_Tire Rotation,CLASS_DESC_Valves
0,277902102.0,0,0,1,1,1,10.0,2.0,1.0,1,1,2016-01-17,9.0,4.0,437.76,2,0,0.024105,0,0.000023,0,0.006679,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,261419318.0,0,0,1,1,1,5.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.076643,0,0.001021,0,0.038920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,279504183.0,0,0,1,1,1,16.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.028503,0,0.001486,0,0.066980,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0
3,261790895.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.033559,0,0.007031,0,0.006402,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,313827641.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,6,0,0.014084,0,0.000089,0,0.000097,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
year_indivs = year_indivs.merge(dy_tire_specs, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [158]:
year_indivs.shape

(3001193, 74)

In [159]:
del dy_tire_specs
gc.collect()

40

In [160]:
# year_indivs.to_csv(path + 'curated_data/year_indivs.csv')

## Feature: Acceptable percntage of miles within a state

The higher the percentage, the more suited the road for driving

In [161]:
dy_state_zip = dy[['INDIV_ID','STATE_CODE', 'ZIP_CODE']]

dy_state_zip.columns = ['indiv','STATE_CODE', 'ZIP_CODE']

In [162]:
dy_state_zip.drop_duplicates(subset = 'indiv', inplace = True)

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [163]:
year_indivs = year_indivs.merge(dy_state_zip, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [164]:
accept_perc = pd.read_csv(path + 'supp_data/curated_supp_data/accept_miles.csv', index_col = 0)

In [165]:
accept_perc.head()

,Code,Percent Acceptable
3,AL,0.861377
4,AK,0.775208
5,AZ,0.803321
6,AR,0.912018
7,CA,0.550144


In [166]:
accept_perc.columns = ['STATE_CODE', 'accept_perc']

In [167]:
accept_perc.drop_duplicates(subset = 'STATE_CODE', inplace = True)

year_indivs = year_indivs.merge(accept_perc, left_on = 'STATE_CODE', right_on = 'STATE_CODE', how = 'left')

In [168]:
del dy_state_zip
del accept_perc
del dy 

gc.collect()

100

## Feature: weather per state

In [169]:
weather_state = pd.read_csv(path + 'supp_data/curated_supp_data/weather_state.csv', index_col = 0)

In [170]:
weather_state.head()

,state,year,month,avg_temp,avg_max_temp,avg_prcp,avg_snow_depth
0,AK,2015,1,17.971178,72.990438,2.379781,475.543727
1,AK,2015,2,20.759009,49.222243,3.012146,345.913970
2,AK,2015,3,22.334774,71.193844,2.818260,492.077742
3,AK,2015,4,31.860780,42.486386,3.655609,503.802364
4,AK,2015,5,46.103190,64.287331,2.700007,734.360541


I will not use time series. Thus, I will use the average across all years

In [171]:
weather_state = weather_state.groupby('state').agg({'avg_temp' : 'mean', 'avg_max_temp' : 'max',
                                  'avg_prcp': 'mean', 'avg_snow_depth': 'mean'}).reset_index()

In [172]:
weather_state = weather_state.rename(columns = {'state' : 'STATE_CODE'})

In [173]:
year_indivs = year_indivs.merge(weather_state, left_on = 'STATE_CODE', right_on = 'STATE_CODE', how = 'left')

In [174]:
del weather_state

gc.collect()

20

In [175]:

# accept_perc.drop_duplicates(subset = 'STATE_CODE', inplace = True)

## Feature: average miles covered

In [176]:
miles_covered = pd.read_csv(path + 'supp_data/curated_supp_data/zip_county.csv', index_col = 0)

In [177]:
miles_covered.head()

,ZIP,est_vmiles
0,19904,45.302813
1,19980,45.302813
2,19952,45.302813
3,19962,45.302813
4,19901,45.302813


In [178]:
miles_covered = miles_covered.rename(columns = {'ZIP' : 'ZIP_CODE'})

miles_covered.drop_duplicates(subset = ['ZIP_CODE'], inplace = True)

In [179]:
year_indivs = year_indivs.merge(miles_covered, left_on = 'ZIP_CODE', right_on = 'ZIP_CODE', how = 'left')

In [180]:
year_indivs.shape

(3001193, 82)

In [181]:
del miles_covered

gc.collect()

40

In [182]:
# ddf = dd.from_pandas(year_indivs, npartitions=10) ## parallelizing using dask

In [183]:
# ddf.to_csv(path + 'curated_data/sample_data-*.csv', index=False)

In [184]:
# year_indivs.to_csv(path + 'curated_data/sample_data-*.csv'
#          , sep='|'
#          , header=True
#          , index=False
#          , chunksize=100000
#          , compression='gzip'
#          , encoding='utf-8')

In [185]:
year_indivs.to_csv(path + 'curated_data/year16.csv')